# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам

<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

В этом проекте мы будем решать задачу идентификации пользователя по его поведению в сети Интернет. Это сложная и интересная задача на стыке анализа данных и поведенческой психологии. В качестве примера, компания Яндекс решает задачу идентификации взломщика почтового ящика по его поведению. В двух словах, взломщик будет себя вести не так, как владелец ящика: он может не удалять сообщения сразу по прочтении, как это делал хозяин, он будет по-другому ставить флажки сообщениям и даже по-своему двигать мышкой. Тогда такого злоумышленника можно идентифицировать и "выкинуть" из почтового ящика, предложив хозяину войти по SMS-коду. Этот пилотный проект описан в [статье](https://habrahabr.ru/company/yandex/blog/230583/) на Хабрахабре. Похожие вещи делаются, например, в Google Analytics и описываются в научных статьях, найти можно многое по фразам "Traversal Pattern Mining" и "Sequential Pattern Mining".


Мы будем решать похожую задачу: по последовательности из нескольких веб-сайтов, посещенных подряд один и тем же человеком, мы будем идентифицировать этого человека. Идея такая: пользователи Интернета по-разному переходят по ссылкам, и это может помогать их идентифицировать (кто-то сначала в почту, потом про футбол почитать, затем новости, контакт, потом наконец – работать, кто-то – сразу работать).

Будем использовать данные из [статьи](http://ceur-ws.org/Vol-1703/paper12.pdf) "A Tool for Classification of Sequential Data". И хотя мы не можем рекомендовать эту статью (описанные методы делеки от state-of-the-art, лучше обращаться к [книге](http://www.charuaggarwal.net/freqbook.pdf) "Frequent Pattern Mining" и последним статьям с ICDM), но данные там собраны аккуратно и представляют интерес.

Имеются данные с прокси-серверов Университета Блеза Паскаля, они имеют очень простой вид. Для каждого пользователя заведен csv-файл с названием user\*\*\*\*.csv (где вместо звездочек – 4 цифры, соответствующие ID пользователя), а в нем посещения сайтов записаны в следующем формате: <br>

<center>*timestamp, посещенный веб-сайт*</center>

Скачать исходные данные можно по ссылке в статье, там же описание.
Для этого задания хватит данных не по всем 3000 пользователям, а по 10 и 150. [Ссылка](https://drive.google.com/file/d/1AU3M_mFPofbfhFQa_Bktozq_vFREkWJA/view?usp=sharing) на архив *capstone_user_identification* (~7 Mb, в развернутом виде ~ 60 Mb). 

В финальном проекте уже придется столкнуться с тем, что не все операции можно выполнить за разумное время (скажем, перебрать с кросс-валидацией 100 комбинаций параметров случайного леса на этих данных Вы вряд ли сможете), поэтому мы будем использовать параллельно 2 выборки: по 10 пользователям и по 150. Для 10 пользователей будем писать и отлаживать код, для 150 – будет рабочая версия. 

Данные устроены следующем образом:

 - В каталоге 10users лежат 10 csv-файлов с названием вида "user[USER_ID].csv", где [USER_ID] – ID пользователя;
 - Аналогично для каталога 150users – там 150 файлов;
 - В каталоге 3users – игрушечный пример из 3 файлов, это для отладки кода предобработки, который Вы далее напишете.


# <center> Подготовка данных к анализу и построению моделей

Первая часть проекта посвящена подготовке данных для дальнейшего описательного анализа и построения прогнозных моделей. Надо будет написать код для предобработки данных (исходно посещенные веб-сайты указаны для каждого пользователя в отдельном файле) и формирования единой обучающей выборки. Также в этой части мы познакомимся с разреженным форматом данных (матрицы `Scipy.sparse`), который хорошо подходит для данной задачи. 

**План:**
 - Часть 1. Подготовка обучающей выборки
 - Часть 2. Работа с разреженным форматом данных

In [2]:

%load_ext watermark

In [3]:
%watermark -v -m -p numpy,scipy,pandas,matplotlib,statsmodels,sklearn -g

CPython 3.7.0
IPython 6.5.0

numpy 1.15.1
scipy 1.1.0
pandas 0.23.4
matplotlib 2.2.3
statsmodels 0.9.0
sklearn 0.19.2

compiler   : MSC v.1912 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 142 Stepping 9, GenuineIntel
CPU cores  : 4
interpreter: 64bit
Git hash   :


In [4]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import pickle
#pip install tqdm
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

In [5]:
from collections import Counter
from  more_itertools import unique_everseen

**Посмотрим на один из файлов с данными о посещенных пользователем (номер 31) веб-страницах.**

In [6]:
# путь к данным
PATH_TO_DATA = 'final\capstone_user_identification'

In [7]:
user31_data = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                       '10users/user0031.csv'))

In [8]:
user31_data.head()

,timestamp,site
0,2013-11-15 08:12:07,fpdownload2.macromedia.com
1,2013-11-15 08:12:17,laposte.net
2,2013-11-15 08:12:17,www.laposte.net
3,2013-11-15 08:12:17,www.google.com
4,2013-11-15 08:12:18,www.laposte.net


**Поставим задачу классификации: идентифицировать пользователя по сессии из 10 подряд посещенных сайтов. Объектом в этой задаче будет сессия из 10 сайтов, последовательно посещенных одним и тем же пользователем, признаками – индексы этих 10 сайтов (чуть позже здесь появится "мешок" сайтов, подход Bag of Words). Целевым классом будет id пользователя.**

### <center>Пример для иллюстрации</center>
**Пусть пользователя всего 2, длина сессии – 2 сайта.**

<center>user0001.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:01</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:11</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:16</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:20</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

<center>user0002.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">00:00:02</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
  <tr>
    <td class="tg-yw4l">00:00:14</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:17</td>
    <td class="tg-031e">facebook.com</td>
  </tr>
  <tr>
    <td class="tg-031e">00:00:25</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

Идем по 1 файлу, нумеруем сайты подряд: vk.com – 1, google.com – 2 и т.д. Далее по второму файлу. 

Отображение сайтов в их индесы должно получиться таким:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">site</th>
    <th class="tg-yw4l">site_id</th>
  </tr>
  <tr>
    <td class="tg-yw4l">vk.com</td>
    <td class="tg-yw4l">1</td>
  </tr>
  <tr>
    <td class="tg-yw4l">google.com</td>
    <td class="tg-yw4l">2</td>
  </tr>
  <tr>
    <td class="tg-yw4l">yandex.ru</td>
    <td class="tg-yw4l">3</td>
  </tr>
  <tr>
    <td class="tg-yw4l">facebook.com</td>
    <td class="tg-yw4l">4</td>
  </tr>
</table>

Тогда обучающая выборка будет такой (целевой признак – user_id):
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-s6z2{text-align:center}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-hgcj{font-weight:bold;text-align:center}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-hgcj">session_id</th>
    <th class="tg-hgcj">site1</th>
    <th class="tg-hgcj">site2</th>
    <th class="tg-amwm">user_id</th>
  </tr>
  <tr>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">2</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">2</td>
  </tr>
  <tr>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">2</td>
  </tr>
</table>

Здесь 1 объект – это сессия из 2 посещенных сайтов 1-ым пользователем (target=1). Это сайты vk.com и google.com (номер 1 и 2). И так далее, всего 4 сессии. Пока сессии у нас не пересекаются по сайтам, то есть посещение каждого отдельного сайта относится только к одной сессии.

## Часть 1. Подготовка обучающей выборки
Реализуем функцию *prepare_train_set*, которая принимает на вход путь к каталогу с csv-файлами *path_to_csv_files* и параметр *session_length* – длину сессии, а возвращает 2 объекта:
- DataFrame, в котором строки соответствуют уникальным сессиям из *session_length* сайтов, *session_length* столбцов – индексам этих *session_length* сайтов и последний столбец – ID пользователя
- частотный словарь сайтов вида {'site_string': [site_id, site_freq]}, например для недавнего игрушечного примера это будет {'vk.com': (1, 2), 'google.com': (2, 2), 'yandex.ru': (3, 3), 'facebook.com': (4, 1)}


In [9]:
def prepare_train_set(path_to_csv_files, session_length=10):
    users=[]
    for_df=[]
    wordcount = {}
    os.chdir(path_to_csv_files) 
    allFiles = glob.glob('user*.csv')
    for file_ in allFiles:
        with open(file_) as f:
            user_id=file_.split('.csv')[0].lstrip('user0')
            for row in f.readlines(): 
                word=row.split(',')[1].strip()
                if word!='site':
                    users.append(user_id)
                    for_df.append(word)
                    if word not in wordcount:
                        wordcount[word] = 1
                    else:
                        wordcount[word] = wordcount[word]+1
            if len(for_df)%session_length!=0:
                for_df.extend(np.zeros(session_length-len(for_df)%session_length).astype('int'))
    val_1=sorted(wordcount.values(), reverse=True)
    val_2=list(range(1,len(wordcount)+1))
    keys=sorted(wordcount, key=wordcount.get, reverse=True)
    vocabulary=dict(zip(keys,zip(val_1,val_2)))
    for_df=list(map(lambda x: vocabulary[x][1] if x!=0 else 0,for_df))
    df=pd.DataFrame(np.array(for_df).reshape(int(len(for_df)/session_length),session_length), 
               columns=list(map(lambda x: 'site'+str(x), range(0,session_length))))
    users=np.array(users)
    users_df=[]
    for user in list(unique_everseen(users)):
        t=users[users==user]
        if len(t)%session_length==0:
            users_df.extend([user]*int((len(t)/session_length)))
        else:
            users_df.extend([user]*int((len(t)/session_length)+1))
    df['user_id']=users_df
    return df,vocabulary

In [10]:
test_df, test_vocab=prepare_train_set('C:\\Users\\wild-moray\\Desktop\\ML\\final\\capstone_user_identification\\150users')

In [11]:
train_data_150, site_freq_150users = prepare_train_set('C:\\Users\\wild-moray\\Desktop\\ML\\final\\capstone_user_identification\\150users', 
                                                     session_length=10)

In [12]:
train_data_3, site_freq_3users = prepare_train_set('C:\\Users\\wild-moray\\Desktop\\ML\\final\\capstone_user_identification\\3users', 
                                                     session_length=10)

In [13]:
train_data_10, site_freq_10users = prepare_train_set('C:\\Users\\wild-moray\\Desktop\\ML\\final\\capstone_user_identification\\10users', 
                                                     session_length=10)

**Примените полученную функцию к игрушечному примеру, убедитесь, что все работает как надо.**

In [14]:
!cat $PATH_TO_DATA/3users/user0001.csv

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
!cat $PATH_TO_DATA/3users/user0002.csv

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
!cat $PATH_TO_DATA/3users/user0003.csv

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [17]:
train_data_toy, site_freq_3users = prepare_train_set(os.path.join(PATH_TO_DATA, '3users'), 
                                                     session_length=10)

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'final\\capstone_user_identification\\3users'

Частоты сайтов (второй элемент кортежа) точно должны быть такими, нумерация может быть любой (первые элементы кортежей могут отличаться).

In [18]:
site_freq_3users

{'google.com': (9, 1),
 'oracle.com': (8, 2),
 'vk.com': (3, 3),
 'meduza.io': (3, 4),
 'mail.google.com': (2, 5),
 'football.kulichki.ru': (2, 6),
 'geo.mozilla.org': (1, 7),
 'accounts.google.com': (1, 8),
 'apis.google.com': (1, 9),
 'plus.google.com': (1, 10),
 'yandex.ru': (1, 11)}

In [23]:
#site_freq_150users
import operator
sorted_x = sorted(site_freq_150users.items(), key=operator.itemgetter(1))


In [24]:
sorted_x[-10:]

[('www.youtube.com', (16319, 10)),
 ('safebrowsing-cache.google.com', (17960, 9)),
 ('plus.google.com', (18467, 8)),
 ('mail.google.com', (19072, 7)),
 ('clients1.google.com', (25087, 6)),
 ('s.youtube.com', (29102, 5)),
 ('apis.google.com', (29983, 4)),
 ('www.facebook.com', (39002, 3)),
 ('www.google.com', (51320, 2)),
 ('www.google.fr', (64785, 1))]

**Для дальнейшего анализа запишем полученные объекты DataFrame в csv-файлы.**

In [25]:
PATH_TO_DATA='C:\\Users\\wild-moray\\Desktop\\ML\\final'
train_data_10.to_csv(os.path.join(PATH_TO_DATA, 
                                       'train_data_10users.csv'), 
                        index_label='session_id', float_format='%d')
train_data_150.to_csv(os.path.join(PATH_TO_DATA, 
                                        'train_data_150users.csv'), 
                         index_label='session_id', float_format='%d')

## Часть 2. Работа с разреженным форматом данных

Если так подумать, то полученные признаки *site1*, ..., *site10* смысла не имеют как признаки в задаче классификации. А вот если воспользоваться идеей мешка слов из анализа текстов – это другое дело. Создадим новые матрицы, в которых строкам будут соответствовать сессии из 10 сайтов, а столбцам – индексы сайтов. На пересечении строки $i$ и столбца $j$ будет стоять число $n_{ij}$ – cколько раз сайт $j$ встретился в сессии номер $i$. Делать это будем с помощью разреженных матриц Scipy – [csr_matrix](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.sparse.csr_matrix.html). Прочитайте документацию, разберитесь, как использовать разреженные матрицы и создайте такие матрицы для наших данных. Сначала проверьте на игрушечном примере, затем примените для 10 и 150 пользователей. 

Обратите внимание, что в коротких сессиях, меньше 10 сайтов, у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить. 

In [26]:
X_toy, y_toy = train_data_3.iloc[:, :-1].values, train_data_3.iloc[:, -1].values

In [27]:
X_toy

array([[ 3,  2,  2,  7,  2,  1,  8,  5,  9, 10],
       [ 3,  1,  1,  1,  0,  0,  0,  0,  0,  0],
       [ 3,  2,  6,  6,  2,  0,  0,  0,  0,  0],
       [ 4,  1,  2,  1,  2,  1,  1,  5, 11,  4],
       [ 4,  1,  2,  0,  0,  0,  0,  0,  0,  0]])

In [155]:
def sparse_matrix(X):
    import gc
    indptr = [0]
    indices = []
    data = []
    for row in X:
        for i in row:
            if i!=0:
                indices.append(i)
                data.append(1)
        indptr.append(len(indices))
    #csr_matrix((data, indices, indptr), dtype=int).toarray()
    result = csr_matrix((data, indices, indptr), dtype=int)#.toarray().todense()[:,1:]
    return result


In [116]:
X_sparse_toy = sparse_matrix(X_toy)

In [117]:
X_sparse_toy

<5x12 sparse matrix of type '<class 'numpy.int32'>'
	with 32 stored elements in Compressed Sparse Row format>

**Размерность разреженной матрицы должна получиться равной 11, поскольку в игрушечном примере 3 пользователя посетили 11 уникальных сайтов.**

In [31]:
X_sparse_toy

matrix([[1, 3, 1, 0, 1, 0, 1, 1, 1, 1, 0],
        [3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 2, 1, 0, 0, 2, 0, 0, 0, 0, 0],
        [4, 2, 0, 2, 1, 0, 0, 0, 0, 0, 1],
        [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [154]:
X_10users, y_10users = train_data_10.iloc[:, :-1].values, \
                       train_data_10.iloc[:, -1].values
X_150users, y_150users = train_data_150.iloc[:, :-1].values, \
                         train_data_150.iloc[:, -1].values

In [156]:
X_150users.shape

(137019, 10)

In [67]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [142]:
%%memit
X_sparse_10users = sparse_matrix(X_10users)

peak memory: 1366.22 MiB, increment: -256.62 MiB


In [158]:
X_sparse_150users = None
X_sparse_150users = sparse_matrix(X_150users)

In [163]:
X_sparse_150users.shape[0]

137019

In [159]:
X_sparse_10users = None
X_sparse_10users = sparse_matrix(X_10users)

**Сохраним эти разреженные матрицы с помощью [pickle](https://docs.python.org/2/library/pickle.html) (сериализация в Python), также сохраним вектора *y_10users, y_150users* – целевые значения (id пользователя)  в выборках из 10 и 150 пользователей. То что названия этих матриц начинаются с X и y, намекает на то, что на этих данных мы будем проверять первые модели классификации.
Наконец, сохраним также и частотные словари сайтов для 3, 10 и 150 пользователей.**

In [ ]:
PATH_TO_DATA='C:\\Users\\wild-moray\\Desktop\\ML\\final'

In [160]:
with open(os.path.join(PATH_TO_DATA, 'X_sparse_10users.pkl'), 'wb') as X10_pkl:
    pickle.dump(X_sparse_10users, X10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'y_10users.pkl'), 'wb') as y10_pkl:
    pickle.dump(y_10users, y10_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_sparse_150users.pkl'), 'wb') as X150_pkl:
    pickle.dump(X_sparse_150users, X150_pkl, protocol=4)
with open(os.path.join(PATH_TO_DATA, 'y_150users.pkl'), 'wb') as y150_pkl:
    pickle.dump(y_150users, y150_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_3users.pkl'), 'wb') as site_freq_3users_pkl:
    pickle.dump(site_freq_3users, site_freq_3users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_10users.pkl'), 'wb') as site_freq_10users_pkl:
    pickle.dump(site_freq_10users, site_freq_10users_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'site_freq_150users.pkl'), 'wb') as site_freq_150users_pkl:
    pickle.dump(site_freq_150users, site_freq_150users_pkl, protocol=2)

In [149]:
with open(os.path.join(PATH_TO_DATA, 'X_sparse_toy.pkl'), 'wb') as X_sparse_toy_pkl:
    pickle.dump(X_sparse_toy, X_sparse_toy_pkl, protocol=2)

In [153]:
with open(os.path.join(PATH_TO_DATA, 'X_sparse_toy.pkl'), 'rb') as X_sparse_toy_pkl:
    r = None
    r = pickle.load(X_sparse_toy_pkl)
td = None
td = r.todense()[:,1:]
td

matrix([[1, 3, 1, 0, 1, 0, 1, 1, 1, 1, 0],
        [3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 2, 1, 0, 0, 2, 0, 0, 0, 0, 0],
        [4, 2, 0, 2, 1, 0, 0, 0, 0, 0, 1],
        [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

**Чисто для подстраховки проверим, что число столбцов в разреженных матрицах `X_sparse_10users` и `X_sparse_150users` равно ранее посчитанным числам уникальных сайтов для 10 и 150 пользователей соответственно.**

In [ ]:
with open(os.path.join(PATH_TO_DATA, 'X_sparse_150users.pkl'), 'wb') as X150_pkl:
    pickle.dump(X_sparse_150users, X150_pkl, protocol=4)
with open(os.path.join(PATH_TO_DATA, 'y_150users.pkl'), 'wb') as y150_pkl:
    pickle.dump(y_150users, y150_pkl, protocol=2)

In [ ]:
assert X_sparse_150users.shape[1] == len(site_freq_150users)